In [1]:
import zipfile
import csv
import pandas as pd
import os
import logging
import pathlib
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gdp
import plotly.express as px 
import plotly.io as pio
import plotly.figure_factory as ff
import kaleido
import glob

from os.path import exists
from pathlib import Path

/opt/homebrew/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
#Subset of Master Data 
if not os.path.exists('exported'):
    os.makedirs('exported')

#Subset with Conditions    
if not os.path.exists('conditions'):
    os.makedirs('conditions')

In [3]:
#Logging File for Metrics
logging.basicConfig(filename='logs.log', level=logging.ERROR)

#Text File for Unit Tests
UnitTest = open(pathlib.Path(".", "UnitTest.txt").resolve(),"w")
#UnitTest = open(r"C:\Users\KEVAL\Desktop\DataEngineering_CapstoneProject\UnitTest.txt", "w")

UnitTest.write("This is the UNIT Test File"+ "\n")

27

In [4]:
#Check if file is in folder

def filecheck(location):
    # check if it exist
    try:
        file_exists = os.path.exists(location)
        path = os.path.dirname(os.path.abspath(location))
        #Print to UnitTest
        UnitTest.write("Successfull!! -- The path of the file is: " + path + "-->" + location + "\n")
    except Exception as e:
        #Print to UnitTest.txt
        UnitTest.write("ERROR: Unsuccessfull!! -- The path of the file is: " + path + "-->" + location + "\n")
        print(e)
        print("Error: File NOT IN LOCATION " + location)
        return
    return 

Download data from source and store locally. The file is then inzipped so the data can be viewed. 

In [5]:
!kaggle datasets download --force -d felix4guti/traffic-violations-in-usa
print('downloaded data')
with zipfile.ZipFile("./{}.zip".format("traffic-violations-in-usa"),"r") as z:
    z.extractall(".")
print('unzipped the data')

  0%|                                               | 0.00/48.5M [00:00<?, ?B/s]

  2%|▊                                     | 1.00M/48.5M [00:00<00:05, 9.29MB/s]

  8%|███▏                                  | 4.00M/48.5M [00:00<00:02, 19.7MB/s]

 14%|█████▍                                | 7.00M/48.5M [00:00<00:01, 23.6MB/s]

 21%|███████▊                              | 10.0M/48.5M [00:00<00:01, 25.6MB/s]

...

 89%|█████████████████████████████████▋    | 43.0M/48.5M [00:01<00:00, 27.3MB/s]

 95%|████████████████████████████████████  | 46.0M/48.5M [00:01<00:00, 27.7MB/s]


100%|██████████████████████████████████████| 48.5M/48.5M [00:01<00:00, 26.5MB/s]


downloaded data


unzipped the data


The dataset is opened. 

In [6]:
#Filter out data without geolocation (Longitude and Latitude)
Process_Data = pd.read_csv('Traffic_Violations.csv')
Process_Data = Process_Data.dropna(subset=['Longitude', 'Latitude'])

#Process_Data = Process_Data[Process_Data['State'].isin(['WA']) ] 

#Save dataframe as new csv
Process_Data.to_csv(pathlib.Path(".", "Traffic_Violations_Final.csv").resolve())
#Process_Data.to_csv(r'C:\Users\KEVAL\Desktop\DataEngineering_CapstoneProject\Traffic_Violations_Final.csv')

#Show Process_Data
display(Process_Data)

/var/folders/_0/q_sx58p90kx486zxqr8s93tm0000gn/T/ipykernel_35074/3692936169.py:2: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  Process_Data = pd.read_csv('Traffic_Violations.csv')


,Date Of Stop,Time Of Stop,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,Belts,...,Charge,Article,Contributed To Accident,Race,Gender,Driver City,Driver State,DL State,Arrest Type,Geolocation
1,12/20/2012,00:41:00,MCP,"2nd district, Bethesda",DRIVING WHILE IMPAIRED BY ALCOHOL,NORFOLK AVE / ST ELMO AVE,38.983578,-77.093105,No,No,...,21-902(b1),Transportation Article,No,WHITE,M,DERWOOD,MD,MD,A - Marked Patrol,"(38.9835782, -77.09310515)"
2,07/20/2012,23:12:00,MCP,"5th district, Germantown",FAILURE TO STOP AT STOP SIGN,WISTERIA DR @ WARING STATION RD,39.161810,-77.253581,No,No,...,21-707(a),Transportation Article,No,ASIAN,F,GERMANTOWN,MD,MD,A - Marked Patrol,"(39.1618098166667, -77.25358095)"
3,03/19/2012,16:10:00,MCP,"2nd district, Bethesda",DRIVER USING HANDS TO USE HANDHELD TELEPHONE W...,CLARENDON RD @ ELM ST. N/,38.982731,-77.100755,No,No,...,21-1124.2(d2),Transportation Article,No,HISPANIC,M,ARLINGTON,VA,VA,A - Marked Patrol,"(38.9827307333333, -77.1007551666667)"
4,12/01/2014,12:52:00,MCP,"6th district, Gaithersburg / Montgomery Village",FAILURE STOP AND YIELD AT THRU HWY,CHRISTOPHER AVE/MONTGOMERY VILLAGE AVE,39.162888,-77.229088,No,No,...,21-403(b),Transportation Article,No,BLACK,F,UPPER MARLBORO,MD,MD,A - Marked Patrol,"(39.1628883333333, -77.2290883333333)"
5,06/09/2012,21:19:00,MCP,"3rd district, Silver Spring",OCCUPANT UNDER 16 NOT RESTRAINED BY SEATBELT,2068 HARLEQUIN TERRACE,39.069143,-76.969678,No,No,...,22-412.3(b),Transportation Article,No,WHITE,F,SILVER SPRING,MD,MD,A - Marked Patrol,"(39.06914295, -76.9696780666667)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018629,12/22/2016,23:27:00,MCP,"2nd district, Bethesda",WINDOW TINT,2445 LYTTONSVILLE RD,38.999260,-77.052685,No,No,...,61,NaN,No,BLACK,F,SILVER SPRING,MD,MD,A - Marked Patrol,"(38.99926, -77.052685)"
1018630,12/20/2016,12:01:00,MCP,"3rd district, Silver Spring",STOP LIGHTS (*),UNIVERSITY BLVD E / E. MELBOURNE AVE,39.010112,-77.000582,No,No,...,64*,NaN,No,BLACK,F,SILVER SPRING,MD,MD,A - Marked Patrol,"(39.0101116666667, -77.0005816666667)"
1018631,12/21/2016,02:11:00,MCP,"4th district, Wheaton",STOP LIGHTS (*),GEORGIA AVE / RANDOLPH RD,39.058475,-77.049772,No,Yes,...,64*,NaN,No,BLACK,F,UPPER MARLBORO,MD,MD,A - Marked Patrol,"(39.058475, -77.0497716666667)"
1018632,12/26/2016,08:07:00,MCP,"4th district, Wheaton",HEADLIGHTS (*),GEORGIA AVE/DENNIS AVE,39.025122,-77.046552,No,No,...,55*,NaN,No,HISPANIC,M,SILVER SPRING,MD,MD,A - Marked Patrol,"(39.0251216666667, -77.0465516666667)"


In [7]:
class Dataset:
    def __init__(self, csv_filename):
        self.dataset = pd.read_csv(csv_filename, dtype='unicode')
        filecheck(csv_filename)
    
    def __generateMetrics(self, subset, columns):
        numberOfRows = len(subset)
        numberOfColumns = len(columns)
        output = "Number of Rows: {}\n".format(numberOfRows) + "Number of Columns: {}\n".format(numberOfColumns) 
        return output
        
    def generate_subset(self, table_name, columns):
        test_data = self.dataset[columns]
        try:
            path = os.path.join('exported', table_name)
            test_data.to_csv(path+'.csv')
            #Check if csv file was made
            filecheck(path+'csv')
            with open(path+'_metrics.txt', 'w') as file:
                file.write(self.__generateMetrics(test_data, columns))
                #Check if the metrics file was made
                filecheck(path+'_metrics.txt')
        except Exception as e:
            logging.error(e)
            print(e)
            return
        return test_data.head()


In [8]:
traffic_data = Dataset("Traffic_Violations_Final.csv")

In [9]:
traffic_data.generate_subset('Incident', ['Arrest Type', 'Description', 'State', 'Violation Type', 'Charge', 'Article', 'Contributed To Accident', 'Longitude', 'Latitude'])

,Arrest Type,Description,State,Violation Type,Charge,Article,Contributed To Accident,Longitude,Latitude
0,A - Marked Patrol,DRIVING WHILE IMPAIRED BY ALCOHOL,MD,Citation,21-902(b1),Transportation Article,No,-77.09310515,38.9835782
1,A - Marked Patrol,FAILURE TO STOP AT STOP SIGN,MD,Citation,21-707(a),Transportation Article,No,-77.25358095,39.1618098166667
2,A - Marked Patrol,DRIVER USING HANDS TO USE HANDHELD TELEPHONE W...,VA,Citation,21-1124.2(d2),Transportation Article,No,-77.1007551666667,38.9827307333333
3,A - Marked Patrol,FAILURE STOP AND YIELD AT THRU HWY,MD,Citation,21-403(b),Transportation Article,No,-77.2290883333333,39.1628883333333
4,A - Marked Patrol,OCCUPANT UNDER 16 NOT RESTRAINED BY SEATBELT,MD,Citation,22-412.3(b),Transportation Article,No,-76.9696780666667,39.06914295


In [10]:
traffic_data.generate_subset('Report', ['HAZMAT', 'Commercial License', 'Fatal', 'Property Damage', 'Personal Injury', 'Belts', 'Accident','Longitude', 'Latitude'])

,HAZMAT,Commercial License,Fatal,Property Damage,Personal Injury,Belts,Accident,Longitude,Latitude
0,No,No,No,No,No,No,No,-77.09310515,38.9835782
1,No,No,No,No,No,No,No,-77.25358095,39.1618098166667
2,No,No,No,No,No,No,No,-77.1007551666667,38.9827307333333
3,No,No,No,Yes,No,No,No,-77.2290883333333,39.1628883333333
4,No,No,No,No,No,No,No,-76.9696780666667,39.06914295


In [11]:
traffic_data.generate_subset('Time', ['Date Of Stop', 'Time Of Stop', 'Location', 'Latitude', 'Longitude', 'Geolocation'])

,Date Of Stop,Time Of Stop,Location,Latitude,Longitude,Geolocation
0,12/20/2012,00:41:00,NORFOLK AVE / ST ELMO AVE,38.9835782,-77.09310515,"(38.9835782, -77.09310515)"
1,07/20/2012,23:12:00,WISTERIA DR @ WARING STATION RD,39.1618098166667,-77.25358095,"(39.1618098166667, -77.25358095)"
2,03/19/2012,16:10:00,CLARENDON RD @ ELM ST. N/,38.9827307333333,-77.1007551666667,"(38.9827307333333, -77.1007551666667)"
3,12/01/2014,12:52:00,CHRISTOPHER AVE/MONTGOMERY VILLAGE AVE,39.1628883333333,-77.2290883333333,"(39.1628883333333, -77.2290883333333)"
4,06/09/2012,21:19:00,2068 HARLEQUIN TERRACE,39.06914295,-76.9696780666667,"(39.06914295, -76.9696780666667)"


In [12]:
traffic_data.generate_subset('Agency', ['Agency','Longitude', 'Latitude'])

,Agency,Longitude,Latitude
0,MCP,-77.09310515,38.9835782
1,MCP,-77.25358095,39.1618098166667
2,MCP,-77.1007551666667,38.9827307333333
3,MCP,-77.2290883333333,39.1628883333333
4,MCP,-76.9696780666667,39.06914295


In [13]:
traffic_data.generate_subset('Subagency', ['SubAgency','Longitude', 'Latitude'])

,SubAgency,Longitude,Latitude
0,"2nd district, Bethesda",-77.09310515,38.9835782
1,"5th district, Germantown",-77.25358095,39.1618098166667
2,"2nd district, Bethesda",-77.1007551666667,38.9827307333333
3,"6th district, Gaithersburg / Montgomery Village",-77.2290883333333,39.1628883333333
4,"3rd district, Silver Spring",-76.9696780666667,39.06914295


In [14]:
traffic_data.generate_subset('Driver', ['DL State', 'Driver State', 'Driver City', 'Gender', 'Race','Longitude', 'Latitude'])

,DL State,Driver State,Driver City,Gender,Race,Longitude,Latitude
0,MD,MD,DERWOOD,M,WHITE,-77.09310515,38.9835782
1,MD,MD,GERMANTOWN,F,ASIAN,-77.25358095,39.1618098166667
2,VA,VA,ARLINGTON,M,HISPANIC,-77.1007551666667,38.9827307333333
3,MD,MD,UPPER MARLBORO,F,BLACK,-77.2290883333333,39.1628883333333
4,MD,MD,SILVER SPRING,F,WHITE,-76.9696780666667,39.06914295


In [15]:
traffic_data.generate_subset('Vehicle', ['VehicleType', 'Make', 'Year', 'Model', 'Color','Longitude', 'Latitude'])

,VehicleType,Make,Year,Model,Color,Longitude,Latitude
0,02 - Automobile,AUDI,2005.0,4S,GRAY,-77.09310515,38.9835782
1,02 - Automobile,TOYT,2002.0,4S,RED,-77.25358095,39.1618098166667
2,02 - Automobile,HONDA,1996.0,CIVIC,SILVER,-77.1007551666667,38.9827307333333
3,02 - Automobile,HONDA,2001.0,ACCORD,SILVER,-77.2290883333333,39.1628883333333
4,02 - Automobile,CHEVROLET,2004.0,IMPALA,SILVER,-76.9696780666667,39.06914295


In [16]:
#Plotting the Data

In [17]:
# Takes in File, Plots Data, and Saves Result File              
class Plotting:
    
    def __init__(self, csv_filename):
        
        file = csv_filename
    
        #Direct Path 
        data_path = pathlib.Path(".", file).resolve()
        df = pd.read_csv(data_path, engine = 'python') 
        
        #Folder path
        #data_path = pathlib.Path("./exported", file).resolve()
        #df = pd.read_csv(data_path, engine = 'python')
        self.CreateMap(df, file)
        return 
    
    #Map the dataframe
    def CreateMap(self, dataframe, filename):
        file = filename
        df_geo = gdp.GeoDataFrame(dataframe, geometry = gdp.points_from_xy(dataframe.Longitude,dataframe.Latitude))
    
        try:     
        
            #Plot Map of USA with Cities 
            us_cities = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/us-cities-top-1k.csv")
            #fig = px.scatter_mapbox(us_cities, lat="lat", lon="lon", color_discrete_sequence=["fuchsia"], zoom=3, height=500)

            #Data Plot
            fig = px.scatter_mapbox(df_geo,lat='Latitude', lon='Longitude', title="Map of Violations", color_discrete_sequence=["fuchsia"], height=750)
            fig.update_layout(mapbox_style="open-street-map")
            fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
          
            #Write Data to file ---- NEEDS TO PRINT WITH TABLE NAME
            pio.write_html(fig,file+".html")
            #pio.write_html(fig,"figure/fig1.html")  
            
            filecheck(file+'.html')
            
            #Create Unique File for each query
            #path = os.path.join('figure', table_name)
            #fig.to_html(path+'.html')
                        
        except Exception as e:
            logging.error(e)
            print(e)
            return
    
        return  
    
    

In [18]:
Plotting("Traffic_Violations_Final.csv")
Plotting("./exported/Incident.csv")
Plotting("./exported/Report.csv")
Plotting("./exported/Time.csv")
Plotting("./exported/Agency.csv")
Plotting("./exported/Subagency.csv")
Plotting("./exported/Driver.csv")
Plotting("./exported/Vehicle.csv")

In [19]:
#Plotting Conditional Data
def condplot(location, title, condition):
    df = pd.read_csv(location, dtype='unicode')
    df = eval(condition)
    
    # write output
    path = os.path.join('conditions', title)
    df.to_csv(path+'.csv')

    #Create and Save Graph
    Plotting("./conditions/"+title+".csv")

In [20]:
#Only Impared by alcohol in Virginia who contributed to accident
condplot("./exported/Incident.csv", "VirginiaAlcoholAccident", "df[(df['Description'].str.contains('Alcohol')) & (df['State'] =='VA')& (df['Contributed To Accident'] =='YES')]" )

In [21]:
#Only Fatal with property damage not wearing a seatbelt
condplot("./exported/Report.csv", "NoSeatBelt_Fatal", "df[(df['Fatal'] == 'YES') & (df['Property Damage'] =='YES') & (df['Belts'] =='NO') ]" )

In [22]:
#Only 2014 between 8am and 5pm
condplot("./exported/Time.csv", "2014_8am_to_8pm", "df[(df['Date Of Stop'] == '2014') & (df['Time Of Stop'] >'08:00:00') & (df['Time Of Stop'] <'17:00:00') ]" )

In [23]:
#Only vehicles that are honda and silver 
condplot("./exported/Agency.csv", "AgencyMCP", "df[(df['Agency'] == 'MCP')]" )

In [24]:
#Only 2nd district Bethesda
condplot("./exported/Subagency.csv", "AgencyofBethesda", "df[(df['SubAgency'] == '2nd district, Bethesda')]" )

In [25]:
#Only Females with New York Drivers License
condplot("./exported/Driver.csv", "NewYorkFemaleDrivers", "df[(df['Driver State'] == 'NY') & (df['Gender'] =='F')]" )

In [26]:
#Only vehicles that are honda and silver 
condplot("./exported/Vehicle.csv", "SilverHondas", "df[(df['Make'] == 'HONDA') & (df['Color'] =='SILVER')]" )